<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220906.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [54]:
from google.colab import drive
import os
import json
from tqdm import tqdm
import shutil
import re
from glob import glob

# 데이터셋 다운로드

In [55]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/trash/dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/trash/dataset


In [44]:
!pwd

/content/drive/MyDrive/공유드라이브/dataset


In [45]:
# yolov5 모델 다운로드
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12190, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 12190 (delta 31), reused 26 (delta 6), pack-reused 12124
Receiving objects: 100% (12190/12190), 12.63 MiB | 11.55 MiB/s, done.
Resolving deltas: 100% (8376/8376), done.


In [46]:
%cd yolov5
!pip install -r requirements.txt

/content/drive/MyDrive/공유드라이브/dataset/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import yaml
import torch
from IPython.display import Image, clear_output

In [57]:
%cd /content/drive/MyDrive/trash/dataset/yolov5
!pwd

/content/drive/MyDrive/trash/dataset/yolov5
/content/drive/MyDrive/trash/dataset/yolov5


# **데이터 셋 준비**


In [61]:
from glob import glob

train_img_lst = glob('/content/drive/MyDrive/trash/dataset/dataset/imgaes/*.jpg')
with open('/content/drive/MyDrive/trash/dataset/dataset/train.txt','w') as f:
  f.write('\n'.join(train_img_lst)+'\n')

In [62]:
print(len(train_img_lst))

1660


In [63]:
from IPython.core.magic import register_line_cell_magic

# register_line_cell_magic을 이용해 IPython에서 바로 사용할 수 있게 정의
# writetimplate를 쉽게 사용할 수 있음
@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [65]:
!cat '/content/drive/MyDrive/trash/dataset/data.yaml'


train: ./dataset/dataset/img
val: ./dataset/dataset/img

nc: 7
names: ['polygon', 'styrofoam', 'glass', 'metal', 'pet_bottle', 'buoy', 'etc']


In [68]:
%%writetemplate /content/drive/MyDrive/trash/dataset/dataset/data.yaml

train: /content/drive/MyDrive/trash/dataset/dataset/imgaes
val: /content/drive/MyDrive/trash/dataset/dataset/imgaes

nc: 7
names: ['polygon', 'styrofoam', 'glass', 'metal', 'pet_bottle', 'buoy', 'etc']

In [69]:
%cat /content/drive/MyDrive/trash/dataset/dataset/data.yaml



train: /content/drive/MyDrive/trash/dataset/dataset/imgaes
val: /content/drive/MyDrive/trash/dataset/dataset/imgaes

nc: 7
names: ['polygon', 'styrofoam', 'glass', 'metal', 'pet_bottle', 'buoy', 'etc']


In [70]:
import yaml
with open('/content/drive/MyDrive/trash/dataset/dataset/data.yaml', 'r') as stream:
  num_classes=str(yaml.safe_load(stream)['nc'])

  print(num_classes)

7


In [71]:
!cat /content/drive/MyDrive/trash/dataset/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [72]:
%%writetemplate /content/drive/MyDrive/trash/dataset/yolov5/models/costom_yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [73]:
!cat /content/drive/MyDrive/trash/dataset/yolov5/models/costom_yolov5s.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 7  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [74]:
!pwd

/content/drive/MyDrive/trash/dataset/yolov5


In [75]:
# 데이터 학습|
!python train.py --img 640 --batch 32 --epochs 100 --data /content/drive/MyDrive/trash/dataset/dataset/data.yaml --cfg /content/drive/MyDrive/trash/dataset/yolov5/models/costom_yolov5s.yaml --weight '' --name ocean_results --cache

train: weights=, cfg=/content/drive/MyDrive/trash/dataset/yolov5/models/costom_yolov5s.yaml, data=/content/drive/MyDrive/trash/dataset/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=ocean_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-94-g1aea74c Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, bo